In [1]:
using Flux

┌ Info: CUDAdrv.jl failed to initialize, GPU functionality unavailable (set JULIA_CUDA_SILENT or JULIA_CUDA_VERBOSE to silence or expand this message)
└ @ CUDAdrv /home/edoardo/.julia/packages/CUDAdrv/mCr0O/src/CUDAdrv.jl:69


In [2]:
versioninfo()

Julia Version 1.3.0
Commit 46ce4d7933 (2019-11-26 06:09 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i7-6820HQ CPU @ 2.70GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, skylake)


In [3]:
mutable struct MinimalRNNCell{T,H}
    kernel::T
    recurrent_kernel::T
    hidden::H
end

function MinimalRNNCell(input_shape, units::Integer)
    kernel = rand(input_shape[end], units)*0.1.-0.05
    recurrent_kernel = rand(units, units)*0.1.-0.05
    MinimalRNNCell(kernel, recurrent_kernel, zeros(units))
end

function (c::MinimalRNNCell)(hᵢ₋₁::AbstractArray, x::AbstractArray)
    output = (c.kernel * x) + (c.recurrent_kernel * hᵢ₋₁)
    c.hidden = output
    return output, output
end
Flux.hidden(c::MinimalRNNCell) = c.hidden

Flux.@functor MinimalRNNCell

In [4]:
function GenerateSample(length, output)
    inputs = randn(length, output) * 0.1
    outputs = zeros((length,output))
    for i in 1:length
        if i == 1
            outputs[i,:] = inputs[i,:]
        else
            outputs[i,:] = inputs[i,:] + inputs[i-1,:] 
        end
    end
    inputs, outputs
end

GenerateSample (generic function with 1 method)

In [5]:
train_x = zeros(50,10,3)
train_y = zeros(50,10,3)
for i in 1:50
   train_x[i,:,:], train_y[i,:,:] = GenerateSample(10,3) 
end

In [6]:
opt = Flux.ADAM()
function loss(model, x,y)
    total_loss = 0.
    for i in 1:size(x,1)
        for j in 1:size(x,2)
            @inbounds total_loss += Flux.mse(model(train_x[i,j,:]),y[i,j,:])
        end    
        Flux.reset!(model)
    end
    total_loss / size(x,1)
end
m = Chain(Flux.Recur(MinimalRNNCell((10,3),3)))

Chain(Recur(MinimalRNNCell{Array{Float64,2},Array{Float64,1}}([0.03687930734524261 0.02754005515463638 -0.024762755556102925; -0.03724291563801674 -0.03712461584111522 0.005946200009111703; -0.004632328710569976 0.04672460924357984 -0.034375428046249115], [0.03963564520571042 0.04668264902157114 0.02846412526055718; -0.04441128115876012 -0.03668172694461785 -0.029210850691534597; 0.035634119136127504 -0.01114081932601603 -0.029229659728985214], [0.0, 0.0, 0.0])))

In [7]:
# Set up hidden state
m(train_x[1,1,:])
@code_warntype m.layers[1].cell(m.layers[1].state, train_x[1,1,:])

Variables
  c::MinimalRNNCell{Array{Float64,2},Array{Float64,1}}
  hᵢ₋₁::Array{Float64,1}
  x::Array{Float64,1}
  output::Array{Float64,1}

Body::Tuple{Array{Float64,1},Array{Float64,1}}
1 ─ %1 = Base.getproperty(c, :kernel)::Array{Float64,2}
│   %2 = (%1 * x)::Array{Float64,1}
│   %3 = Base.getproperty(c, :recurrent_kernel)::Array{Float64,2}
│   %4 = (%3 * hᵢ₋₁)::Array{Float64,1}
│        (output = %2 + %4)
│        Base.setproperty!(c, :hidden, output)
│   %7 = Core.tuple(output, output)::Tuple{Array{Float64,1},Array{Float64,1}}
└──      return %7


In [9]:
function fit!(model, train_x, train_y, epochs; opt=Flux.ADAM(), ps=params(model) )
    Flux.reset!(model)
    for i in 1:epochs
        println("Epoch $i")
         start = time_ns()
        gs = gradient(ps) do
            loss(model, train_x, train_y)
        end
        Flux.Optimise.update!(opt, ps, gs)
        stop = time_ns()
        println("$((stop-start) / 1000 ) μs")
    end
end

fit! (generic function with 1 method)

In [10]:
fit!(m, train_x, train_y, 10)

Epoch 1
5.825926645e6 μs
Epoch 2
65054.949 μs
Epoch 3
72434.712 μs
Epoch 4
62050.673 μs
Epoch 5
67654.452 μs
Epoch 6
70062.746 μs
Epoch 7
86605.568 μs
Epoch 8
71512.929 μs
Epoch 9
76081.85 μs
Epoch 10
61641.571 μs
